In [1]:
# Transformers installation
! pip install pyarrow==14.0.2 transformers[torch] datasets evaluate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00


# Pretraining


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/UW/CSE\ 447\ \(NLP\)/Final\ Project

/content/gdrive/MyDrive/UW/CSE 447 (NLP)/Final Project


In [ ]:
!ls

 data					   OpenBookQA_base.ipynb   pretrain-obqa.ipynb
'Final Project Methods and Results.gdoc'   openbookqa_base_model


In [ ]:
!rm -rf pretrained-obqa-model

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForMultipleChoice,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    set_seed,
)
from datasets import load_dataset
import evaluate
import numpy as np
import math
import os

In [7]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
text_datasets = {
    "train": ['data/obqa_facts_train.txt'],
    "eval": ['data/obqa_facts_val.txt'],
    "test": ['data/obqa_facts_test.txt']
}
dataset = load_dataset("text", data_files=text_datasets)

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Tokenizes each batch of lines in the dataset, just truncate to 100
# no line in pretraining corpus is longer. Also, don't pad preemptively
tokenized_dataset = dataset.map(
    lambda z: tokenizer(z["text"], truncation=True, max_length=100),
    num_proc=2,
    batched=True,
    remove_columns=["text"],
)

Map (num_proc=2):   0%|          | 0/4957 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
BATCH_SIZE = 96
STEPS_PER_EPOCH = math.ceil(len(tokenized_dataset["train"]) / BATCH_SIZE)
NUM_EPOCHS = 20
print(STEPS_PER_EPOCH)

52


In [ ]:
model = AutoModelForMaskedLM.from_pretrained("roberta-base")
set_seed(42)

args = TrainingArguments(
    output_dir="pretrained-obqa-model",
    overwrite_output_dir=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=192,
    num_train_epochs=NUM_EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    weight_decay=0.01,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"]
)

In [ ]:
trainer.train()  # batch_size=96, lr=2e-5

Epoch,Training Loss,Validation Loss
1,1.723500,1.314612
2,1.378900,1.095075
3,1.203300,1.120169
4,1.074900,0.812587
5,1.013800,0.904303
6,0.933600,0.780677
7,0.868400,0.711803
8,0.805200,0.810355
9,0.772400,0.626402
10,0.714600,0.621052


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=1040, training_loss=0.8251945477265578, metrics={'train_runtime': 822.6223, 'train_samples_per_second': 120.517, 'train_steps_per_second': 1.264, 'total_flos': 1288457684886042.0, 'train_loss': 0.8251945477265578, 'epoch': 20.0})

In [ ]:
!cp -r pretrained-obqa-model gdrive/MyDrive

## Fine-tuning

In [3]:
openbookqa = load_dataset("openbookqa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
openbookqa['train'][:2]

{'id': ['7-980', '7-584'],
 'question_stem': ['The sun is responsible for',
  'When standing miles away from Mount Rushmore'],
 'choices': [{'text': ['puppies learning new tricks',
    'children growing up and getting old',
    'flowers wilting in a vase',
    'plants sprouting, blooming and wilting'],
   'label': ['A', 'B', 'C', 'D']},
  {'text': ['the mountains seem very close',
    'the mountains are boring',
    'the mountains look the same as from up close',
    'the mountains seem smaller than in photographs'],
   'label': ['A', 'B', 'C', 'D']}],
 'answerKey': ['D', 'D']}

The preprocessing function you want to create needs to:

1. Make four copies of the `sent1` field and combine each of them with `sent2` to recreate how a sentence starts.
2. Combine `sent2` with each of the four possible sentence endings.
3. Flatten these two lists so you can tokenize them, and then unflatten them afterward so each example has a corresponding `input_ids`, `attention_mask`, and `labels` field.

In [8]:
def preprocess_function_qa(examples):
    qstems = [[context] * 4 for context in examples["question_stem"]]
    choices = [[choice for choice in dictionary['text']] for dictionary in examples["choices"]]

    qstems = sum(qstems, [])
    choices = sum(choices, [])

    tokenized_examples = tokenizer(qstems, choices, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

output = preprocess_function_qa(openbookqa['train'][:2])

def preprocess_labels_qa(examples):
    lettermap = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    answers = list(examples["answerKey"])
    answers = [lettermap[a] for a in answers]

    return {'label': answers}

In [9]:
tokenized_qa = openbookqa.map(preprocess_function_qa, batched=True)
tokenized_qa = tokenized_qa.map(preprocess_labels_qa, batched=True)

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_qa['train'][0].keys())

dict_keys(['id', 'question_stem', 'choices', 'answerKey', 'input_ids', 'attention_mask', 'label'])


🤗 Transformers doesn't have a data collator for multiple choice, so you'll need to adapt the [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding) to create a batch of examples. It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

`DataCollatorForMultipleChoice` flattens all the model inputs, applies padding, and then unflattens the results:

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        # print(features)
        # print(features[0])
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [10]:
BATCH_SIZE = 32
NUM_EPOCHS = 10
STEPS_PER_EPOCH = math.ceil(len(tokenized_qa["train"]) / BATCH_SIZE)
print(STEPS_PER_EPOCH)

155


In [ ]:
PRETRAINED_MODEL_PATH = "pretrained-obqa-model/checkpoint-936"
model = AutoModelForMultipleChoice.from_pretrained(PRETRAINED_MODEL_PATH)
set_seed(42)

training_args = TrainingArguments(
    output_dir="pt-obqa-ft-obqa-model",
    overwrite_output_dir=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qa["train"],
    eval_dataset=tokenized_qa["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at pretrained-obqa-model/checkpoint-936 and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.203900,1.116582,0.510000
2,0.884300,1.076799,0.582000
3,0.594400,1.214697,0.566000
4,0.366600,1.492364,0.568000
5,0.243200,1.715602,0.570000
6,0.159000,1.837963,0.580000
7,0.121200,1.993272,0.576000
8,0.072400,2.269535,0.564000
9,0.057300,2.378068,0.554000
10,0.042300,2.483166,0.550000


TrainOutput(global_step=1550, training_loss=0.3744557312996157, metrics={'train_runtime': 1424.1438, 'train_samples_per_second': 34.807, 'train_steps_per_second': 1.088, 'total_flos': 4248738784998072.0, 'train_loss': 0.3744557312996157, 'epoch': 10.0})

In [ ]:
trainer.evaluate(tokenized_qa["test"], metric_key_prefix="test")

{'test_loss': 1.1280643939971924,
 'test_accuracy': 0.534,
 'test_runtime': 5.1214,
 'test_samples_per_second': 97.63,
 'test_steps_per_second': 3.124,
 'epoch': 10.0}

In [ ]:
!cp -r pt-obqa-ft-obqa-model/ gdrive/MyDrive